In [13]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 23: Crab Cups ---</h2><p>The small crab challenges <em>you</em> to a game! The crab is going to mix up some cups, and you have to predict where they'll end up.</p>
<p>The cups will be arranged in a circle and labeled <em>clockwise</em> (your puzzle input). For example, if your labeling were <code>32415</code>, there would be five cups in the circle; going clockwise around the circle from the first cup, the cups would be labeled <code>3</code>, <code>2</code>, <code>4</code>, <code>1</code>, <code>5</code>, and then back to <code>3</code> again.</p>
<p>Before the crab starts, it will designate the first cup in your list as the <em>current cup</em>. The crab is then going to do <em>100 moves</em>.</p>
<p>Each <em>move</em>, the crab does the following actions:</p>
<ul>
<li>The crab picks up the <em>three cups</em> that are immediately <em>clockwise</em> of the <em>current cup</em>. They are removed from the circle; cup spacing is adjusted as necessary to maintain the circle.</li>
<li>The crab selects a <em>destination cup</em>: the cup with a <em>label</em> equal to the <em>current cup's</em> label minus one. If this would select one of the cups that was just picked up, the crab will keep subtracting one until it finds a cup that wasn't just picked up. If at any point in this process the value goes below the lowest value on any cup's label, it <em>wraps around</em> to the highest value on any cup's label instead.</li>
<li>The crab places the cups it just picked up so that they are <em>immediately clockwise</em> of the destination cup. They keep the same order as when they were picked up.</li>
<li>The crab selects a new <em>current cup</em>: the cup which is immediately clockwise of the current cup.</li>
</ul>
<p>For example, suppose your cup labeling were <code>389125467</code>. If the crab were to do merely 10 moves, the following changes would occur:</p>
<pre><code>-- move 1 --
cups: (3) 8  9  1  2  5  4  6  7 
pick up: 8, 9, 1
destination: 2

-- move 2 --
cups: 3 (2) 8 9 1 5 4 6 7
pick up: 8, 9, 1
destination: 7

-- move 3 --
cups: 3 2 (5) 4 6 7 8 9 1
pick up: 4, 6, 7
destination: 3

-- move 4 --
cups: 7 2 5 (8) 9 1 3 4 6
pick up: 9, 1, 3
destination: 7

-- move 5 --
cups: 3 2 5 8 (4) 6 7 9 1
pick up: 6, 7, 9
destination: 3

-- move 6 --
cups: 9 2 5 8 4 (1) 3 6 7
pick up: 3, 6, 7
destination: 9

-- move 7 --
cups: 7 2 5 8 4 1 (9) 3 6
pick up: 3, 6, 7
destination: 8

-- move 8 --
cups: 8 3 6 7 4 1 9 (2) 5
pick up: 5, 8, 3
destination: 1

-- move 9 --
cups: 7 4 1 5 8 3 9 2 (6)
pick up: 7, 4, 1
destination: 5

-- move 10 --
cups: (5) 7 4 1 8 3 9 2 6
pick up: 7, 4, 1
destination: 3

-- final --
cups: 5 (8) 3 7 4 1 9 2 6
</code></pre>

<p>In the above example, the cups' values are the labels as they appear moving clockwise around the circle; the <em>current cup</em> is marked with <code>( )</code>.</p>
<p>After the crab is done, what order will the cups be in? Starting <em>after the cup labeled <code>1</code></em>, collect the other cups' labels clockwise into a single string with no extra characters; each number except <code>1</code> should appear exactly once. In the above example, after 10 moves, the cups clockwise from <code>1</code> are labeled <code>9</code>, <code>2</code>, <code>6</code>, <code>5</code>, and so on, producing <em><code>92658374</code></em>. If the crab were to complete all 100 moves, the order after cup <code>1</code> would be <em><code>67384529</code></em>.</p>
<p>Using your labeling, simulate 100 moves. <em>What are the labels on the cups after cup <code>1</code>?</em></p>
</article>


In [14]:
from collections import deque
from itertools import islice


example = "389125467"


def crabcubs(s: str, moves: int = 100, do_print=False) -> str:
    cups = deque([int(i) for i in s])

    mx = max(cups)

    for move in range(1, moves + 1):
        if do_print:
            print(f"-- move {move} --")
            print(
                f"cups: {" ".join(f"{f"({c})" if c == current else f"{c}"}" for c in cups)}"
            )

        current = cups.popleft()
        pickup = [cups.popleft(), cups.popleft(), cups.popleft()]

        destination = (current - 2) % mx + 1
        while destination in pickup:
            destination = (destination - 2) % mx + 1

        if do_print:
            print(f"pick up:  {" ".join(map(str,pickup))}")
            print(f"destination: {destination}")
            print()

        rotate_destination = cups.index(destination)
        cups.rotate(-rotate_destination)
        cups.popleft()
        cups.extendleft(reversed(pickup))
        cups.appendleft(destination)
        cups.rotate(rotate_destination)
        cups.appendleft(current)
        cups.rotate(-1)

    if do_print:
        print("-- final --")
        print(
            f"cups: {" ".join(f"{f"({c})" if c == current else f"{c}"}" for c in cups)}"
        )
    cups.rotate(-cups.index(1))
    return "".join(str(i) for i in islice(cups, 1, len(cups)))


assert crabcubs(example, 10) == "92658374"
assert crabcubs(example) == "67384529"

In [15]:
puzzle = "942387615"

print(f"Part I: {crabcubs(puzzle)}")

Part I: 36542897


<link href="style.css" rel="stylesheet"></link>
<main class="read-aloud">

<p>Your puzzle answer was <code>36542897</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Due to what you can only assume is a mistranslation (you're <span title="If I were going for a programming language pun here, I'd say you were a little... RUSTy.">not exactly fluent in Crab</span>), you are quite surprised when the crab starts arranging <em>many</em> cups in a circle on your raft - <em>one million</em> (<code>1000000</code>) in total.</p>
<p>Your labeling is still correct for the first few cups; after that, the remaining cups are just numbered in an increasing fashion starting from the number after the highest number in your list and proceeding one by one until one million is reached. (For example, if your labeling were <code>54321</code>, the cups would be numbered <code>5</code>, <code>4</code>, <code>3</code>, <code>2</code>, <code>1</code>, and then start counting up from <code>6</code> until one million is reached.) In this way, every number from one through one million is used exactly once.</p>
<p>After discovering where you made the mistake in translating Crab Numbers, you realize the small crab isn't going to do merely 100 moves; the crab is going to do <em>ten million</em> (<code>10000000</code>) moves!</p>
<p>The crab is going to hide your <em class="star">stars</em> - one each - under the <em>two cups that will end up immediately clockwise of cup <code>1</code></em>. You can have them if you predict what the labels on those cups will be when the crab is finished.</p>
<p>In the above example (<code>389125467</code>), this would be <code>934001</code> and then <code>159792</code>; multiplying these together produces <em><code>149245887792</code></em>.</p>
<p>Determine which two cups will end up immediately clockwise of cup <code>1</code>. <em>What do you get if you multiply their labels together?</em></p>
</article>

</main>


In [16]:
@dataclass
class Node:
    value: int
    previous: None | Node = None
    next: None | Node = None


class CacheLinkedList:
    def __init__(self, cups: list[int]) -> None:
        n = len(cups)
        cache = [None] * (n + 1)

        for cup in cups:
            cache[cup] = Node(cup)

        for i in range(n):
            cache[cups[i]].next = cache[cups[(i + 1) % n]]
            cache[cups[i]].previous = cache[cups[(i - 1) % n]]

        self.n = n
        self.cache = cache
        self.cups = cups

    def to_list(self, node: Node) -> list[int]:
        result = [node.value]
        current = node.next
        while current.value != node.value:
            result.append(current.value)
            current = current.next
        return result

    def __getitem__(self, value: int) -> Node:
        return self.cache[value]

    def __len__(self) -> int:
        return self.n

    def __repr__(self) -> str:
        return "".join(map(str, self.to_list(self[self.cups[0]])))


def crabcups(s: str, moves=100, length=None) -> CacheLinkedList:
    cups = [int(i) for i in s]

    if length is not None:
        cups += list(range(len(cups) + 1, length + 1))

    cll = CacheLinkedList(cups)
    n = len(cll)

    current = cll[cups[0]]
    for _ in range(moves):
        current = cll[current.value]

        pickup_first = current.next
        pickup_last = pickup_first.next.next
        pickup_values = [pickup_first.value, pickup_first.next.value, pickup_last.value]

        destination = (current.value - 2) % n + 1
        while destination in pickup_values:
            destination = (destination - 2) % n + 1

        destination = cll[destination]

        current.next = pickup_last.next
        pickup_last.next.previous = current

        pickup_last.next = destination.next
        destination.next.previous = pickup_last
        destination.next = pickup_first
        pickup_first.previous = destination

        current = current.next

    return cll


def part_I(s, moves=100) -> str:
    cll = crabcups(s, moves)
    return "".join(map(str, cll.to_list(cll[1])[1:]))


def part_II(s) -> str:
    cll = crabcups(s, moves=10_000_000, length=1_000_000)
    return cll[1].next.value * cll[1].next.next.value


assert part_I(example, moves=10) == "92658374"
assert part_I(example, moves=100) == "67384529"
assert part_I(puzzle, moves=100) == "36542897"

assert part_II(example) == 149245887792

print(f"Part II: {part_II(puzzle)}")

Part II: 562136730660


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>562136730660</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


In [17]:
from more_itertools import minmax

# A lot faster ;)


def fast_dic(cups, steps):  # non-rotated dictionary:  {cup: nextcup, ... }
    # Solution from buji: https://www.reddit.com/r/adventofcode/comments/kimluc/2020_day_23_solutions/
    current = cups[0]
    mn, mx = minmax(cups)
    carousel = dict(zip(cups, cups[1:] + [cups[0]]))

    for _ in range(steps):
        xyz = [carousel[current]]
        xyz.append(carousel[xyz[0]])
        xyz.append(carousel[xyz[-1]])
        dest = current - 1

        while dest < mn or dest in xyz:
            dest -= 1
            if dest < mn:
                dest = mx

        carousel[current] = carousel[xyz[-1]]
        carousel[xyz[-1]] = carousel[dest]
        carousel[dest] = xyz[0]
        current = carousel[current]

    return carousel


def crabcups1(s: str):
    cups = fast_dic([int(i) for i in s], 100)
    labels = [cups[1]]
    while labels[-1] != 1:
        labels.append(cups[labels[-1]])
    return "".join([str(c) for c in labels[:-1]])


print("Part I:", crabcups1(puzzle))


def crabcups2(s: str):
    cups = [int(i) for i in s]
    cups += list(range(max(cups) + 1, 1_000_001))
    cups = fast_dic(cups, 10_000_000)
    c1 = cups[1]
    c2 = cups[c1]
    return c1 * c2


print("Part II:", crabcups2(puzzle))

Part I: 36542897
Part II: 562136730660
